In [4]:
from cycler import cycler
from process_requestlogs import *
from create_window_statistics import *
from gather_memtier_statistics import *
import matplotlib.pyplot as plt

In [5]:
LOG_INTERVAL = 20 # Every 20th request is logged
WINDOWSIZE_SEC = 5 # The window aggregates over 5 seconds. For the throughput (ops/sec) we need to divide by 5 again
windowsize = str(WINDOWSIZE_SEC) + 's'

In [6]:
from create_graphs import *
inputdir = "/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816"
middlewares = ["middleware_04"]
dir_suffix_regex_string = "_\d*vc\d*workers"
warmup_period_endtime = 10
cooldown_period_starttime = 72
outdir = "./graphs"
client_logfiles = ["client_01.log"]
create_workload_graphs(inputdir, "writeOnly", middlewares, client_logfiles, dir_suffix_regex_string, warmup_period_endtime, cooldown_period_starttime, outdir)
create_workload_graphs(inputdir, "readOnly", middlewares, client_logfiles, dir_suffix_regex_string, warmup_period_endtime, cooldown_period_starttime, outdir)

Concatenating requestlogs in /home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/1/middleware_04/2017-11-17 223537


In [ ]:
ping_log_filepath = '/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/1/client_ping_01.log'
startingClockTime=metrics['initializeClockTime'].min()
print(startingClockTime)
ping_log = extract_ping_logs(ping_log_filepath, startingClockTime)

In [ ]:
metrics